# Data exploration

* This is a data exploration notebook. We aim to analyze the data to gain insight on the classification task that consists in predicting whether a particular DNA sequence exhibits favorable properties for bounding a protein.

## Imports

In [6]:
import csv
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data loading

* We investigate directly the *raw* data (i.e. the sequences of *A*, *T*, *C* and *G*) rather than the intermediate representation obtained by using K-Means on sub-sequences.

In [7]:
CWD = os.getcwd()
DATA_DIR = os.path.join(CWD, "data")
RESULT_DIR = os.path.join(CWD, "results")

FILES = {0: {"train_mat": "Xtr0_mat100.csv",
             "train": "Xtr0.csv",
             "test_mat": "Xte0_mat100.csv",
             "test": "Xte0.csv",
             "label": "Ytr0.csv"},
         1: {"train_mat": "Xtr1_mat100.csv",
             "train": "Xtr1.csv",
             "test_mat": "Xte1_mat100.csv",
             "test": "Xte1.csv",
             "label": "Ytr1.csv"},
         2: {"train_mat": "Xtr2_mat100.csv",
             "train": "Xtr2.csv",
             "test_mat": "Xte2_mat100.csv",
             "test": "Xte2.csv",
             "label": "Ytr2.csv"}}

In [8]:
def load_data(file_id, mat=True):
    
    X_train = list()
    Y_train = list()
    X_test = list()
    
    dic = FILES[file_id]
    
    if mat:
        files = [dic["train_mat"], dic["label"], dic["test_mat"]]
    else:
        files = [dic["train"], dic["label"], dic["test"]]

    for file, l in zip(files, [X_train, Y_train, X_test]):
        with open(os.path.join(DATA_DIR, file), "r", newline="") as csvfile:
            if "mat" in file:
                reader = csv.reader(csvfile, delimiter=" ")
                for row in reader:
                    l.append(row)
            else:
                reader = csv.reader(csvfile, delimiter=",")
                next(reader, None) # Skip the header
                for row in reader:
                    l.append(row[1])
                
    if mat:
        X_train = np.array(X_train).astype("float")
        Y_train = np.array(Y_train).astype("int")
        X_test = np.array(X_test).astype("float")
        np.random.seed(0)
        index = np.random.permutation(len(X_train))
        X_train = X_train[index]
        Y_train = Y_train[index]
    
    else:
        np.random.seed(0)
        index = np.random.permutation(len(X_train))
        X_train = [X_train[i] for i in index]
        Y_train = [Y_train[i] for i in index]
        Y_train = np.array(Y_train).astype("int")
        
    
    return X_train, Y_train, X_test

In [9]:
# Instanciate dictionaries
TRAIN_DIC = {i: {"sequence": None, "label": None} for i in range(len(FILES))}
TEST_DIC = {i: {"sequence": None} for i in range(len(FILES))}

# Populate dictionaries
for i in range(len(FILES)):
    X_train, Y_train, X_test = load_data(i, mat=False)
    TRAIN_DIC[i]["sequence"] = X_train
    TRAIN_DIC[i]["label"] = Y_train
    TEST_DIC[i]["sequence"] = X_test

# Populate list of dictionaries to concatenate them below
train_dfs = list()
test_dfs = list()

for i in range(len(FILES)):
    tmp_train = pd.DataFrame(data=TRAIN_DIC[i])
    tmp_train["dataset"] = i # dataset index
    train_dfs.append(tmp_train)
    tmp_test = pd.DataFrame(data=TEST_DIC[i])
    tmp_test["dataset"] = i # dataset index
    test_dfs.append(tmp_test)

# Build pandas dataframe (training set)
train_df = pd.concat(train_dfs, ignore_index=True)
# Build pandas dataframe (testing set)
test_df = pd.concat(test_dfs, ignore_index=True)
train_df.head()

,sequence,label,dataset
0,TTTTTGGAGATGGAATTTCAATCTTGTTGCCCAGGGTGGAATGCAA...,0,0
1,TTTTTTTAGATGGAGTCTCACTCTTGTCGCCCAGGCTGGAGTGCAA...,0,0
2,GTACAACGGAGATAATCATCTGAGCTCTGTCTGCTTCCTCTGGCTA...,1,0
3,TTTCTAGACAGGAAAATTGAAACATATTAGCTTTATTCATGTATAG...,0,0
4,GAAGAATGGCGTGAACCTGGGAGGCAGAGGTTGCAGTGAGCCGAGA...,0,0


In [15]:
def compute_stats(df, test=False):
    """
    Process a dataframe to compute statistics 
    on the sequences present in the three datasets
    
    Parameters
    -----------
    - df : pandas.DataFrame
        Dataframe on which to compute statistics (either train or test)
    
    - test : boolean (optional)    
        True if dealing with test data (i.e. skip duplicates label sanity check)
    
    Returns
    -----------
    - df : pandas.DataFrame
        Original dataframe populated with new statistics
    """

    # Length of the strings
    df["seq_length"] = df.sequence.str.len()

    # Number of A, T, C, G
    df["As"] = df.sequence.str.count("A")
    df["Ts"] = df.sequence.str.count("T")
    df["Cs"] = df.sequence.str.count("C")
    df["Gs"] = df.sequence.str.count("G")
    assert all(df.As + df.Ts + df.Cs + df.Gs == df.seq_length) # There are only A, T, C and G
    df.head()

    # TODO: intra and extra occurences (i.e. how many occurences we have for each sequence in its own dataset VS in all datasets)

    # Extra occurences
    _, inv, c = np.unique(df.sequence.values, return_inverse=True, return_counts=True)
    df["unique_all"] = [c[inv[i]] for i in range(len(inv))]

    if not test:
        # Assert that the sequence multiple occurences have the same label
        doubles_df = df[df.unique_all > 1].sort_values(by="sequence")[["sequence", "label"]]

        values = doubles_df.iloc[0].values
        curr_seq = values[0]
        curr_label = values[1]

        for i in range(1, len(doubles_df)):
            values = doubles_df.iloc[i].values
            next_seq = values[0]
            next_label = values[1]
            if next_seq != curr_seq:
                continue
            else:
                if next_label != curr_label:
                    raise ValueError(f"Duplicate ({next_seq}) has different labels! ")

    # TODO: Intra occurences


    # Longest straight sequence (letter and number)
    for k, seq in enumerate(df.sequence.values):
        counts = {a: 0 for a in ["A", "T", "C", "G"]}
        curr_count = 0
        curr_char = seq[0]
        curr_count += 1
        for i in range(1, len(seq)):
            next_char = seq[i]
            if next_char != curr_char:
                if curr_count > counts[curr_char]:
                    counts[curr_char] = curr_count
                # reset current count
                curr_count = 0
                curr_char = next_char
                curr_count += 1
            else:
                curr_count += 1
        df.loc[k, "A_longest"] = counts["A"]
        df.loc[k, "T_longest"] = counts["T"]
        df.loc[k, "C_longest"] = counts["C"]
        df.loc[k, "G_longest"] = counts["G"]
        df.loc[k, "Longest"] = max(counts.values())
        df.loc[k, "Longest_char"] = max(counts, key=counts.get)

    return df    

In [13]:
train_stats_df = compute_stats(train_df)
train_stats_df.describe()

,label,dataset,seq_length,As,Ts,Cs,Gs,unique_all,A_longest,T_longest,C_longest,G_longest,Longest
count,6000.000000,6000.000000,6000.0,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,0.495667,1.000000,101.0,24.626167,24.407167,26.162333,25.804333,1.070667,3.098500,3.114833,3.137833,3.143000,4.445167
std,0.500023,0.816565,0.0,7.672998,7.793149,8.079637,7.847801,0.256289,1.486438,1.497115,1.033447,1.053756,1.516260
min,0.000000,0.000000,101.0,2.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,101.0,19.000000,19.000000,21.000000,20.000000,1.000000,2.000000,2.000000,2.000000,2.000000,4.000000
50%,0.000000,1.000000,101.0,24.000000,24.000000,26.000000,25.000000,1.000000,3.000000,3.000000,3.000000,3.000000,4.000000
75%,1.000000,2.000000,101.0,30.000000,30.000000,31.000000,31.000000,1.000000,4.000000,4.000000,4.000000,4.000000,5.000000
max,1.000000,2.000000,101.0,56.000000,58.000000,57.000000,56.000000,2.000000,22.000000,24.000000,12.000000,15.000000,24.000000


In [21]:
train_stats_df[train_stats_df.dataset == 0].describe()

,label,dataset,seq_length,As,Ts,Cs,Gs,unique_all,A_longest,T_longest,C_longest,G_longest,Longest
count,2000.000000,2000.0,2000.0,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000
mean,0.488500,0.0,101.0,28.068000,28.163500,22.294500,22.474000,1.011000,3.545000,3.68550,2.838000,2.845000,4.715000
std,0.499993,0.0,0.0,7.291087,7.455707,6.666858,6.490716,0.104329,1.612227,1.75446,0.974798,0.928119,1.804273
min,0.000000,0.0,101.0,6.000000,6.000000,3.000000,3.000000,1.000000,1.000000,1.00000,1.000000,1.000000,2.000000
25%,0.000000,0.0,101.0,23.000000,23.000000,18.000000,18.000000,1.000000,3.000000,3.00000,2.000000,2.000000,4.000000
50%,0.000000,0.0,101.0,28.000000,28.000000,22.000000,22.000000,1.000000,3.000000,3.00000,3.000000,3.000000,4.000000
75%,1.000000,0.0,101.0,33.000000,33.000000,26.000000,27.000000,1.000000,4.000000,4.00000,3.000000,3.000000,5.000000
max,1.000000,0.0,101.0,52.000000,53.000000,55.000000,53.000000,2.000000,22.000000,24.00000,12.000000,7.000000,24.000000


In [22]:
train_stats_df[train_stats_df.dataset == 1].describe()

,label,dataset,seq_length,As,Ts,Cs,Gs,unique_all,A_longest,T_longest,C_longest,G_longest,Longest
count,2000.000000,2000.0,2000.0,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.499000,1.0,101.0,23.172000,23.041000,27.591500,27.195500,1.201000,3.049500,2.989000,3.295500,3.303000,4.416000
std,0.500124,0.0,0.0,7.397823,7.365354,7.811859,7.802289,0.400848,1.417058,1.263201,1.004831,1.093521,1.392449
min,0.000000,1.0,101.0,4.000000,5.000000,7.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,0.000000,1.0,101.0,18.000000,18.000000,22.000000,21.750000,1.000000,2.000000,2.000000,3.000000,3.000000,4.000000
50%,0.000000,1.0,101.0,23.000000,22.000000,27.000000,27.000000,1.000000,3.000000,3.000000,3.000000,3.000000,4.000000
75%,1.000000,1.0,101.0,28.000000,28.000000,33.000000,32.000000,1.000000,4.000000,3.000000,4.000000,4.000000,5.000000
max,1.000000,1.0,101.0,56.000000,58.000000,57.000000,55.000000,2.000000,22.000000,18.000000,8.000000,15.000000,22.000000


In [23]:
train_stats_df[train_stats_df.dataset == 2].describe()

,label,dataset,seq_length,As,Ts,Cs,Gs,unique_all,A_longest,T_longest,C_longest,G_longest,Longest
count,2000.000000,2000.0,2000.0,2000.000000,2000.000000,2000.000000,2000.000000,2000.0,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.499500,2.0,101.0,22.638500,22.017000,28.601000,27.743500,1.0,2.701000,2.670000,3.280000,3.281000,4.204500
std,0.500125,0.0,0.0,7.132689,7.122083,8.214228,8.063822,0.0,1.289741,1.229982,1.053639,1.068461,1.254379
min,0.000000,2.0,101.0,2.000000,2.000000,4.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,2.0,101.0,18.000000,17.000000,23.000000,23.000000,1.0,2.000000,2.000000,3.000000,3.000000,4.000000
50%,0.000000,2.0,101.0,22.000000,21.000000,28.000000,27.000000,1.0,2.000000,2.000000,3.000000,3.000000,4.000000
75%,1.000000,2.0,101.0,27.000000,26.000000,34.000000,33.000000,1.0,3.000000,3.000000,4.000000,4.000000,5.000000
max,1.000000,2.0,101.0,49.000000,48.000000,55.000000,56.000000,1.0,17.000000,17.000000,8.000000,9.000000,17.000000


In [17]:
test_stats_df = compute_stats(test_df, test=True)
test_stats_df.describe()

,dataset,seq_length,As,Ts,Cs,Gs,unique_all,A_longest,T_longest,C_longest,G_longest,Longest
count,3000.000000,3000.0,3000.000000,3000.00000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,1.000000,101.0,24.530000,24.37400,26.047667,26.048333,1.038667,3.060667,3.158667,3.131333,3.138000,4.468333
std,0.816633,0.0,7.655846,7.73098,8.173596,7.994037,0.192831,1.446035,1.621018,1.019034,1.071761,1.583193
min,0.000000,101.0,3.000000,2.00000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,101.0,19.000000,19.00000,20.000000,21.000000,1.000000,2.000000,2.000000,2.000000,2.000000,4.000000
50%,1.000000,101.0,24.000000,24.00000,26.000000,26.000000,1.000000,3.000000,3.000000,3.000000,3.000000,4.000000
75%,2.000000,101.0,30.000000,29.00000,32.000000,31.000000,1.000000,4.000000,4.000000,4.000000,4.000000,5.000000
max,2.000000,101.0,53.000000,56.00000,60.000000,64.000000,2.000000,26.000000,28.000000,8.000000,12.000000,28.000000


In [18]:
test_stats_df[test_stats_df.dataset == 0].describe()

,dataset,seq_length,As,Ts,Cs,Gs,unique_all,A_longest,T_longest,C_longest,G_longest,Longest
count,1000.0,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.0000,1000.000000,1000.000000,1000.000000
mean,0.0,101.0,28.088000,28.247000,22.089000,22.576000,1.006000,3.54800,3.8250,2.809000,2.832000,4.785000
std,0.0,0.0,7.549918,7.211657,6.816858,6.406658,0.077266,1.66387,2.0299,0.922697,0.935228,2.095511
min,0.0,101.0,6.000000,7.000000,5.000000,8.000000,1.000000,1.00000,1.0000,1.000000,1.000000,2.000000
25%,0.0,101.0,23.000000,23.750000,17.000000,18.000000,1.000000,2.00000,3.0000,2.000000,2.000000,4.000000
50%,0.0,101.0,29.000000,28.000000,22.000000,22.000000,1.000000,3.00000,3.0000,3.000000,3.000000,4.000000
75%,0.0,101.0,33.000000,33.000000,26.000000,27.000000,1.000000,4.00000,4.0000,3.000000,3.000000,5.000000
max,0.0,101.0,53.000000,55.000000,55.000000,52.000000,2.000000,26.00000,28.0000,7.000000,7.000000,28.000000


In [19]:
test_stats_df[test_stats_df.dataset == 1].describe()

,dataset,seq_length,As,Ts,Cs,Gs,unique_all,A_longest,T_longest,C_longest,G_longest,Longest
count,1000.0,1000.0,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
mean,1.0,101.0,23.154000,22.92800,27.432000,27.486000,1.110000,2.982000,3.011000,3.293000,3.309000,4.37100
std,0.0,0.0,7.068681,7.20352,7.492898,7.844319,0.313046,1.213734,1.307895,0.976783,1.051963,1.24133
min,1.0,101.0,5.000000,6.00000,9.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.00000
25%,1.0,101.0,18.000000,18.00000,22.000000,22.000000,1.000000,2.000000,2.000000,3.000000,3.000000,4.00000
50%,1.0,101.0,23.000000,22.00000,27.000000,27.000000,1.000000,3.000000,3.000000,3.000000,3.000000,4.00000
75%,1.0,101.0,28.000000,28.00000,32.000000,32.000000,1.000000,3.000000,4.000000,4.000000,4.000000,5.00000
max,1.0,101.0,53.000000,56.00000,57.000000,64.000000,2.000000,14.000000,14.000000,8.000000,8.000000,14.00000


In [20]:
test_stats_df[test_stats_df.dataset == 2].describe()

,dataset,seq_length,As,Ts,Cs,Gs,unique_all,A_longest,T_longest,C_longest,G_longest,Longest
count,1000.0,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,2.0,101.0,22.348000,21.947000,28.622000,28.083000,1.0,2.652000,2.640000,3.292000,3.273000,4.249000
std,0.0,0.0,7.046456,7.247117,8.585151,8.425028,0.0,1.274567,1.149662,1.075124,1.151432,1.197678
min,2.0,101.0,3.000000,2.000000,1.000000,3.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.0,101.0,18.000000,17.000000,23.000000,23.000000,1.0,2.000000,2.000000,3.000000,3.000000,4.000000
50%,2.0,101.0,22.000000,21.000000,28.000000,27.000000,1.0,2.000000,2.000000,3.000000,3.000000,4.000000
75%,2.0,101.0,27.000000,26.000000,34.000000,33.000000,1.0,3.000000,3.000000,4.000000,4.000000,5.000000
max,2.0,101.0,51.000000,48.000000,60.000000,57.000000,1.0,14.000000,9.000000,8.000000,12.000000,14.000000
